<a href="https://colab.research.google.com/github/biruk50/Medium_articles/blob/main/testing_set_covering_algorithms_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This project requires Python 3.7 or above:



In [ ]:
import sys

assert sys.version_info >= (3, 7)

In [ ]:
!pip install pulp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 43.7 MB/s eta 0:00:00


In [ ]:
import timeit
import heapq
import pulp


In [18]:

# Set of all unique stadium areas
stadiums_needed = set([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])# Dictionary of camera locations and the stadium areas they cover

camera_locations = {
    "camera1": set([1, 3, 4, 6, 7]),
    "camera2": set([4, 7, 8, 12]),
    "camera3": set([2, 5, 9, 11, 13]),
    "camera4": set([1, 2, 14, 15]),
    "camera5": set([3, 6, 10, 12, 14]),
    "camera6": set([8, 14, 15]),
    "camera7": set([1, 2, 6, 11]),
    "camera8": set([1, 2, 4, 6, 8, 12])
}

# improved
def improved_set_covering(stadiums_needed, camera_locations):
    # Dictionary to store how many cameras cover each stadium and which cameras
    Count = {}
    final_camera_locations = set()
    # Count occurrences of each stadium in the camera_locations
    for camera, stadiums in camera_locations.items():
        for stadium in stadiums:
            if stadium in Count:
                Count[stadium][0] += 1
                Count[stadium][1].add(camera)
            else:
                Count[stadium] = [1, set([camera])]  # Initialize with count 1 and set of cameras

    # Function to calculate the number of stadiums in common with stadiums_needed
    def stadiums_in_common(camera):
        return len(stadiums_needed & camera_locations[camera])

    # Function to add the selected camera to the final list and update stadiums_needed
    def add_to_final(camera, stadiums_needed):
        if camera in final_camera_locations:
            return
        final_camera_locations.add(camera)
        stadiums_needed -= camera_locations[camera]

    i = 1  # This keeps track of the Count values
    while stadiums_needed:
        min_set =set()
        for stadium, value in Count.items():
            if value[0] == i:
                min_set.update(value[1])  # Add all cameras covering the stadium with count i

        min_set-=final_camera_locations

        if min_set and i > 1:
            best_camera = max(min_set, key=stadiums_in_common)
            min_set.discard(best_camera)
            add_to_final(best_camera, stadiums_needed)

        elif min_set and i == 1:
            add_to_final(min_set.pop(), stadiums_needed)
        else:
            i += 1  # Increase i if no cameras with i coverage

    return final_camera_locations

# Greedy Algorithm
def greedy_set_covering(stadiums_needed, camera_locations):
    final_locations =[]
    while stadiums_needed:
        best_location = None
        area_covered = set()
        for location, stadium_area in camera_locations.items():
            covered = stadiums_needed & stadium_area
            if len(covered) > len(area_covered):
                best_location = location
                area_covered = covered
        if best_location:
            final_locations.append(best_location)
            stadiums_needed -= area_covered
            del camera_locations[best_location]
        else:
            break
    return final_locations

# ILP Algorithm
def ilp_set_covering(stadiums_needed, camera_locations):
    prob = pulp.LpProblem("SetCovering", pulp.LpMinimize)
    location_vars = pulp.LpVariable.dicts("Station", camera_locations.keys(), 0, 1, pulp.LpBinary)
    prob += pulp.lpSum([location_vars[location] for location in camera_locations.keys()])
    for stadium in stadiums_needed:
        prob += pulp.lpSum([location_vars[location] for location in camera_locations.keys() if stadium in camera_locations[location]]) >= 1
    prob.solve()
    selected_locations = [locations for locations in camera_locations.keys() if location_vars[locations].varValue == 1]
    return selected_locations

# Heap-Based Algorithm
def heap_set_covering(stadiums_needed, camera_locations):
    final_locations =[]
    Count = {state: 0 for state in stadiums_needed}

    # Count occurrences of each area in the camera_locations
    for location, stadium_area in camera_locations.items():
        for area in stadium_area:
            if area in Count:
                Count[area] += 1

    heap = []

    # Function to calculate the number of stadium_area in common with stadium_needed
    def area_in_common(stadium_area):
        return len(stadiums_needed & stadium_area)

    # Get the maximum count value
    max_count = max(Count.values(), default=0)

    # Iterate over count values from 1 to max_count
    for count in range(1, max_count + 1):
        for location, stadium_area in camera_locations.items():
            if any(Count[state] == count for state in stadium_area):
                heapq.heappush(heap, (count, -area_in_common(stadium_area), location, stadium_area))

    # Process the heap
    while stadiums_needed and heap:
        _, _, location, stadium_area = heapq.heappop(heap)
        if location in final_locations:
            continue
        final_locations.append(location)
        stadiums_needed -= stadium_area
        for st in stadium_area:
            if st in Count:
                del Count[st]
        del camera_locations[location]

        # Rebuild the heap with remaining areas
        heap = []
        for count in range(1, max_count + 1):
            for location, stadium_area in camera_locations.items():
                if any(area in Count and Count[area] == count for area in stadium_area):
                    heapq.heappush(heap, (count, -area_in_common(stadium_area), location, stadium_area))

    return final_locations

# Measure runtime
def measure_runtime(func, *args):
    start_time = timeit.default_timer()
    result = func(*args)
    end_time = timeit.default_timer()
    return result, end_time - start_time

# Run and measure algorithms
improved_result, improved_time = measure_runtime(improved_set_covering, stadiums_needed.copy(), camera_locations.copy())
greedy_result, greedy_time = measure_runtime(greedy_set_covering, stadiums_needed.copy(), camera_locations.copy())
heap_result, heap_time = measure_runtime(heap_set_covering, stadiums_needed.copy(), camera_locations.copy())
ilp_result, ilp_time = measure_runtime(ilp_set_covering, stadiums_needed.copy(), camera_locations.copy())

# Print results
print(f"Improved Algorithm: {improved_result}, Time: {improved_time}")
print(f"Greedy Algorithm: {greedy_result}, Time: {greedy_time}")
print(f"Heap-Based Algorithm: {heap_result}, Time: {heap_time}")
print(f"ILP Algorithm: {ilp_result}, Time: {ilp_time}")

# Summary
print(f"Improved Algorithm Time: {improved_time}")
print(f"Greedy Algorithm Time: {greedy_time}")
print(f"Heap-Based Algorithm Time: {heap_time}")
print(f"ILP Algorithm Time: {ilp_time}")


Improved Algorithm: {'camera5', 'camera3', 'camera2', 'camera4'}, Time: 5.418799992185086e-05
Greedy Algorithm: ['camera8', 'camera3', 'camera5', 'camera1', 'camera4'], Time: 2.1408000066003297e-05
Heap-Based Algorithm: ['camera3', 'camera5', 'camera1', 'camera6'], Time: 0.00017376600044372026
ILP Algorithm: ['camera2', 'camera3', 'camera4', 'camera5'], Time: 0.013670941000782477
Improved Algorithm Time: 5.418799992185086e-05
Greedy Algorithm Time: 2.1408000066003297e-05
Heap-Based Algorithm Time: 0.00017376600044372026
ILP Algorithm Time: 0.013670941000782477


In [ ]:
states_needed = set(["mt", "wa", "or", "id", "nv", "ut", "ca", "az"])

stations = {
    "kone": set(["id", "nv", "ut"]),
    "ktwo": set(["wa", "id", "mt"]),
    "kthree": set(["or", "nv", "ca"]),
    "kfour":  set(["nv", "ut"]),
    "kfive": set(["ca", "az"])
}
# Greedy Algorithm
def greedy_set_covering(states_needed, stations):
    final_stations = set()
    while states_needed:
        best_station = None
        states_covered = set()
        for station, station_states in stations.items():
            covered = states_needed & station_states
            if len(covered) > len(states_covered):
                best_station = station
                states_covered = covered
        if best_station:
            final_stations.add(best_station)
            states_needed -= states_covered
            del stations[best_station]
        else:
            break
    return final_stations

# ILP Algorithm
def ilp_set_covering(states_needed, stations):
    prob = pulp.LpProblem("SetCovering", pulp.LpMinimize)
    station_vars = pulp.LpVariable.dicts("Station", stations.keys(), 0, 1, pulp.LpBinary)
    prob += pulp.lpSum([station_vars[station] for station in stations.keys()])
    for state in states_needed:
        prob += pulp.lpSum([station_vars[station] for station in stations.keys() if state in stations[station]]) >= 1
    prob.solve()
    selected_stations = [station for station in stations.keys() if station_vars[station].varValue == 1]
    return selected_stations

# Heap-Based Algorithm
def heap_set_covering(states_needed, stations):
    final_stations = set()
    Count = {state: 0 for state in states_needed}

    # Count occurrences of each state in the stations
    for station, station_states in stations.items():
        for state in station_states:
            if state in Count:
                Count[state] += 1

    heap = []

    # Function to calculate the number of states in common with states_needed
    def states_in_common(station_states):
        return len(states_needed & station_states)

    # Get the maximum count value
    max_count = max(Count.values(), default=0)

    # Iterate over count values from 1 to max_count
    for count in range(1, max_count + 1):
        for station, station_states in stations.items():
            if any(Count[state] == count for state in station_states):
                heapq.heappush(heap, (count, -states_in_common(station_states), station, station_states))

    # Process the heap
    while states_needed and heap:
        _, _, station, station_states = heapq.heappop(heap)
        if station in final_stations:
            continue
        final_stations.add(station)
        states_needed -= station_states
        for st in station_states:
            if st in Count:
                del Count[st]
        del stations[station]

        # Rebuild the heap with remaining stations
        heap = []
        for count in range(1, max_count + 1):
            for station, station_states in stations.items():
                if any(state in Count and Count[state] == count for state in station_states):
                    heapq.heappush(heap, (count, -states_in_common(station_states), station, station_states))

    return final_stations

# Measure runtime
def measure_runtime(func, *args):
    start_time = timeit.default_timer()
    result = func(*args)
    end_time = timeit.default_timer()
    return result, end_time - start_time

# Run and measure algorithms
greedy_result, greedy_time = measure_runtime(greedy_set_covering, states_needed.copy(), stations.copy())
heap_result, heap_time = measure_runtime(heap_set_covering, states_needed.copy(), stations.copy())
ilp_result, ilp_time = measure_runtime(ilp_set_covering, states_needed.copy(), stations.copy())

# Print results
print(f"Greedy Algorithm: {greedy_result}, Time: {greedy_time}")
print(f"Heap-Based Algorithm: {heap_result}, Time: {heap_time}")
print(f"ILP Algorithm: {ilp_result}, Time: {ilp_time}")

# Summary
print(f"Greedy Algorithm Time: {greedy_time}")
print(f"Heap-Based Algorithm Time: {heap_time}")
print(f"ILP Algorithm Time: {ilp_time}")

Greedy Algorithm: {'kfive', 'kthree', 'kone', 'ktwo'}, Time: 1.4332999995758655e-05
Heap-Based Algorithm: {'kfive', 'ktwo', 'kfour', 'kthree'}, Time: 7.308799999350413e-05
ILP Algorithm: ['ktwo', 'kthree', 'kfour', 'kfive'], Time: 0.011743835999993735
Greedy Algorithm Time: 1.4332999995758655e-05
Heap-Based Algorithm Time: 7.308799999350413e-05
ILP Algorithm Time: 0.011743835999993735


In [ ]:
states_needed = {"s1", "s2", "s3", "s4", "s5", "s6"}

stations = {
    "A": {"s1", "s2", "s3"},
    "B": {"s1", "s4"},
    "C": {"s2", "s5"},
    "D": {"s3", "s6"},
    "E": {"s4", "s5", "s6"}
}

# Greedy Algorithm
def greedy_set_covering(states_needed, stations):
    final_stations = set()
    while states_needed:
        best_station = None
        states_covered = set()
        for station, station_states in stations.items():
            covered = states_needed & station_states
            if len(covered) > len(states_covered):
                best_station = station
                states_covered = covered
        if best_station:
            final_stations.add(best_station)
            states_needed -= states_covered
            del stations[best_station]
        else:
            break
    return final_stations

# ILP Algorithm
def ilp_set_covering(states_needed, stations):
    prob = pulp.LpProblem("SetCovering", pulp.LpMinimize)
    station_vars = pulp.LpVariable.dicts("Station", stations.keys(), 0, 1, pulp.LpBinary)
    prob += pulp.lpSum([station_vars[station] for station in stations.keys()])
    for state in states_needed:
        prob += pulp.lpSum([station_vars[station] for station in stations.keys() if state in stations[station]]) >= 1
    prob.solve()
    selected_stations = [station for station in stations.keys() if station_vars[station].varValue == 1]
    return selected_stations

# Heap-Based Algorithm
def heap_set_covering(states_needed, stations):
    final_stations = set()
    Count = {state: 0 for state in states_needed}

    # Count occurrences of each state in the stations
    for station, station_states in stations.items():
        for state in station_states:
            if state in Count:
                Count[state] += 1

    heap = []

    # Function to calculate the number of states in common with states_needed
    def states_in_common(station_states):
        return len(states_needed & station_states)

    # Get the maximum count value
    max_count = max(Count.values(), default=0)

    # Iterate over count values from 1 to max_count
    for count in range(1, max_count + 1):
        for station, station_states in stations.items():
            if any(Count[state] == count for state in station_states):
                heapq.heappush(heap, (count, -states_in_common(station_states), station, station_states))

    # Process the heap
    while states_needed and heap:
        _, _, station, station_states = heapq.heappop(heap)
        if station in final_stations:
            continue
        final_stations.add(station)
        states_needed -= station_states
        for st in station_states:
            if st in Count:
                del Count[st]
        del stations[station]

        # Rebuild the heap with remaining stations
        heap = []
        for count in range(1, max_count + 1):
            for station, station_states in stations.items():
                if any(state in Count and Count[state] == count for state in station_states):
                    heapq.heappush(heap, (count, -states_in_common(station_states), station, station_states))

    return final_stations

# Measure runtime
def measure_runtime(func, *args):
    start_time = timeit.default_timer()
    result = func(*args)
    end_time = timeit.default_timer()
    return result, end_time - start_time

# Run and measure algorithms
greedy_result, greedy_time = measure_runtime(greedy_set_covering, states_needed.copy(), stations.copy())
heap_result, heap_time = measure_runtime(heap_set_covering, states_needed.copy(), stations.copy())
ilp_result, ilp_time = measure_runtime(ilp_set_covering, states_needed.copy(), stations.copy())

# Print results
print(f"Greedy Algorithm: {greedy_result}, Time: {greedy_time}")
print(f"Heap-Based Algorithm: {heap_result}, Time: {heap_time}")
print(f"ILP Algorithm: {ilp_result}, Time: {ilp_time}")

# Summary
print(f"Greedy Algorithm Time: {greedy_time}")
print(f"Heap-Based Algorithm Time: {heap_time}")
print(f"ILP Algorithm Time: {ilp_time}")

Greedy Algorithm: {'E', 'A'}, Time: 1.1156000027767732e-05
Heap-Based Algorithm: {'E', 'A'}, Time: 4.9620000027061906e-05
ILP Algorithm: ['A', 'E'], Time: 0.009903937000217411
Greedy Algorithm Time: 1.1156000027767732e-05
Heap-Based Algorithm Time: 4.9620000027061906e-05
ILP Algorithm Time: 0.009903937000217411
